In [35]:
#To install wandb package ,which is helpful in generating plots and report.
!pip install wandb

In [36]:
# Essentials
import numpy as np
import tensorflow
from tensorflow import keras
from keras import regularizers
from keras.models import Sequential
from keras.utils import np_utils
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Flatten, Conv2D, BatchNormalization, Dropout, MaxPooling2D, Activation
import wandb
from wandb.keras import WandbCallback
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from matplotlib import gridspec

import random
import imageio
import os
import cv2
import glob
# random.seed(42)

### Download INaturalist Dataset from github repo

In [37]:
# Load Dataset
!git clone https://github.com/ashwanth10/INaturalist_Dataset.git

fatal: destination path 'INaturalist_Dataset' already exists and is not an empty directory.


## Preprocess Train & Test Data

In [38]:
# Labels for INaturalist Dataset
class_labels = {
    0: 'Amphibia',
    1: 'Animalia',
    2: 'Arachnida',
    3: 'Aves',
    4: 'Fungi',
    5: 'Insecta', 
    6: 'Mammalia', 
    7: 'Mollusca', 
    8: 'Plantae',
    9: 'Reptilia'
}

num_classes = 10
img_size = 128
directory_train = 'INaturalist_Dataset/train'
directory_test  = 'INaturalist_Dataset/val'

# Load training data
x_train = []
y_train = []
for label, name in class_labels.items():
   list_images = os.listdir(directory_train + '/' + name)
   for image_name in list_images:
       try:
          image = imageio.imread(directory_train +'/'+name+'/'+image_name)
       except: 
          continue
       if np.ndim(image) == 3:
          x_train.append(cv2.resize(image, (img_size, img_size)))
          y_train.append(label)
       else: 
          # Some images are black and white
          print(image_name)

# Load testing data
x_test = []
y_test = []
for label, name in class_labels.items():
   list_images = os.listdir(directory_test + '/' + name)
   for image_name in list_images:
       try:
          image = imageio.imread(directory_test +'/'+name+'/'+image_name)
       except: 
          continue
       if np.ndim(image) == 3:
          x_test.append(cv2.resize(image, (img_size, img_size)))
          y_test.append(label)
       else: 
          # Some images are black and white
          print(image_name)

aa66b3ba5ccffdb3c41c1d3acf7dd569.jpg
d192b73b0af85824c65e91f8795d8df7.jpg
2fad1ed3a05629c089cd530f297e27e6.jpg


In [39]:
#converting training list into numpy arrays.
x_train = np.array(x_train)
y_train = np.array(y_train)

#converting test list into numpy arrays.
x_test = np.array(x_test)
y_test = np.array(y_test)


In [40]:
# Set seed
np.random.seed(0)

In [41]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train,
                                                  test_size=0.1, 
                                                  stratify=y_train,
                                                  random_state=42)

In [42]:
# normalize training data
x_train = x_train / 255
x_val = x_val / 255

#normalize testing data
x_test = x_test / 255

In [43]:
# One-hot encoding for training data
targs = np.zeros((len(y_train), 10, 1))
for i in range(len(y_train)):
    targs[i][y_train[i]][0] = 1 # preparing the target matrix 

y_train = targs

In [44]:
# One-hot encoding for validation data
targs_val = np.zeros((len(y_val), 10, 1))
for i in range(len(y_val)):
    targs_val[i][y_val[i]][0] = 1 # preparing the target matrix 

y_val = targs_val

In [45]:
# One-hot encoding for testing data
targs_test = np.zeros((len(y_test), 10, 1))
for i in range(len(y_test)):
    targs_test[i][y_test[i]][0] = 1 # preparing the target matrix 

y_test = targs_test

# Functions and some helper Functions

In [46]:
def addActivation(model, activation_name):
    if activation_name == "relu":
        model.add(Activation('relu'))
    elif activation_name == "elu":
        model.add(Activation('elu'))
    elif activation_name == "selu":
        model.add(Activation('selu'))

In [47]:
def configureBatchNormalization(model, do_batch_normalisation):
    if do_batch_normalisation:
        model.add(BatchNormalization())

In [48]:
def calculateTestAccuracy(model,x_test,y_test):
  yhat=model.predict(x_test)
  yhat_max = np.argmax(yhat, axis=1)
  y_test_max = np.argmax(y_test, axis=1)
  y_test_max = y_test_max.reshape(-1)
  accuracy= (yhat_max == y_test_max).sum()/ y_test.shape[0]
  print('accuracy: ', accuracy)

In [49]:
def train_with_params(x_train, y_train, x_val, y_val, x_test, y_test,
                      filter, kernel_size, input_shape, weight_decay, 
                      activation_name, do_batch_normalisation, neurons_in_dense,
                      dropout, num_classes, learning_rate, batch_size,
                      do_data_augmentation, manual_run=False):

    # clear session
    tensorflow.keras.backend.clear_session()

    # Define the model architecture
    model = Sequential()

    # no. of convolution layers
    NUM_OF_CONV_LAYERS = 5

    for i in range(NUM_OF_CONV_LAYERS):
        model.add(Conv2D(filters = filter[i], kernel_size = kernel_size[i],
                          padding = 'same', input_shape = input_shape, 
                          kernel_regularizer=regularizers.l2(weight_decay)))
        
        addActivation(model, activation_name)
        configureBatchNormalization(model, do_batch_normalisation)
        model.add(MaxPooling2D(pool_size=(2, 2))) 
    
    # Fully connected  layer
    model.add(Flatten())
    model.add(Dense(neurons_in_dense, activation=activation_name, kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Dropout(dropout))
    model.add(BatchNormalization())

    # Output layer
    model.add(Dense(num_classes, activation = "softmax"))

    # Define the optimizer
    optimizer = Adam(lr=learning_rate, beta_1=0.9, beta_2=0.999)
    
    model.compile(loss = "categorical_crossentropy", optimizer = optimizer, metrics=['accuracy'])

    # Data augmentation
    if do_data_augmentation:
        datagen = ImageDataGenerator(
            rotation_range=45,  # randomly rotate images in the range (degrees, 0 to 180)
            width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
            height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
            horizontal_flip=True,  # randomly flip images
            vertical_flip=False  # randomly flip images
        )
    else:
        datagen = ImageDataGenerator(rescale = 1.0)

    datagen.fit(x_train)
    
    # Call Wandb Only when initiated.
    if manual_run:
        model.fit(datagen.flow(x_train, y_train, batch_size=batch_size),
        epochs = 1,
        verbose = 1,
        validation_data= (x_val, y_val))
    else: 
        model.fit(datagen.flow(x_train, y_train, batch_size=batch_size),
        epochs = 1,
        verbose = 1,
        validation_data= (x_val, y_val),
        callbacks = [WandbCallback()])

    # Calculate accuracy of the model
    calculateTestAccuracy(model, x_test, y_test)
    model.save('/content/drive/MyDrive/Models/model.keras')
    
    return model

## WandB Config Params for the Sweeps

In [50]:
sweep_config = {
    'method': 'random', 
    'metric': {
      'name': 'accuracy',
      'goal': 'maximize'   
    },
    'parameters': {
        'kernel_size':{
            'values': [[(3,3),(3,3),(3,3),(3,3),(3,3)], [(3,3),(5,5),(5,5),(7,7),(7,7)], [(7,7),(7,7),(5,5),(5,5),(3,3)], [(3,3),(5,5),(7,7),(9,9),(11,11)] ]
        },
        'weight_decay': {
            'values': [0, 0.0005, 0.005]
        },
        'dropout': {
            'values': [0, 0.2, 0.4]
        },
        'learning_rate': {
            'values': [1e-3, 1e-4]
        },
        'activation': {
            'values': ['relu', 'elu', 'selu']
        },
        'batch_norm':{
            'values': [True, False]
        },
        'filt_org':{
            'values': [[32,32,32,32,32],[32,64,64,128,128],[128,128,64,64,32],[32,64,128,256,512]]
        },
        'data_augment': {
            'values': [True, False]
        },
        'batch_size': {
            'values': [32, 64]
        },
        'num_dense':{
            'values': [64, 128, 256, 512]
        }
    }
}

In [51]:
sweep_id = wandb.sweep(sweep_config, entity="cs21m010-cs21m041", project="DL_Assignment_2")

Create sweep with ID: mi1lqu26
Sweep URL: https://wandb.ai/cs21m010-cs21m041/DL_Assignment_2/sweeps/mi1lqu26


# Run WandB Sweeps

In [52]:
def train():

    # Initialize a new wandb run
    wandb.init(config=sweep_config)
    
    # Config is a variable that holds and saves hyperparameters and inputs
    config = wandb.config
    wandb.run.name = 'num_dense_'+ str(config.num_dense)+'_bs_'+str(config.batch_size)+'_ac_'+ config.activation
    
    # Determine input shape
    input_shape = (img_size, img_size , 3)
    
    train_with_params(x_train, y_train, x_val, y_val, x_test, y_test,
                      config.filt_org, config.kernel_size, input_shape, 
                      config.weight_decay, config.activation, config.batch_norm, 
                      config.num_dense, config.dropout, num_classes, 
                      config.learning_rate, config.batch_size, config.data_augment)
    

In [ ]:
wandb.agent(sweep_id, train, count = 1)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: Agent Starting Run: j9yambrc with config:
wandb: 	activation: selu
wandb: 	batch_norm: True
wandb: 	batch_size: 32
wandb: 	data_augment: True
wandb: 	dropout: 0
wandb: 	filt_org: [128, 128, 64, 64, 32]
wandb: 	kernel_size: [[3, 3], [5, 5], [7, 7], [9, 9], [11, 11]]
wandb: 	learning_rate: 0.001
wandb: 	num_dense: 128
wandb: 	weight_decay: 0.005


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


131/282 [============>.................] - ETA: 28:25 - loss: 4.7711 - accuracy: 0.1899

# Get the Best Model

In [ ]:
best_kernel_size = [(3,3),(3,3),(3,3),(3,3),(3,3)]
best_weight_decay = 0.005
best_dropout = 0.4
best_learning_rate = 0.0001
best_activation = 'elu'
best_batch_size = 64
best_batch_norm = False
best_filt_org = [32,64,128,256,512]
best_data_augment = True
best_num_dense = 128
input_shape = (img_size, img_size , 3)

model = train_with_params(x_train, y_train, x_val, y_val, x_test, y_test,
                      best_filt_org, best_kernel_size, input_shape, 
                      best_weight_decay, best_activation, best_batch_norm, 
                      best_num_dense, best_dropout, num_classes, 
                      best_learning_rate, best_batch_size, best_data_augment, manual_run=True)

# Visualize Data

In [ ]:
vis_test_images = []
vis_true_labels = []
for label, name in class_labels.items():
    list_images = os.listdir(directory_test + '/' + name)

    # Capture 3 random images as demo images
    for j in range(3):
        dummy = random.randint(0, 199)
        image = imageio.imread(directory_test + '/' + name + '/' + list_images[dummy])
        if np.ndim(image) == 3:
            vis_test_images.append(cv2.resize(image, (img_size,img_size)))
            vis_true_labels.append(class_labels[label])

vis_test_images = np.array(vis_test_images)
vis_test_images = vis_test_images / 255

# Plotting the figure
fig = plt.figure(figsize=(60,50))

gs = gridspec.GridSpec(10, 3, width_ratios=[1, 1, 1], wspace=0.0, hspace=1.0, top=0.95, bottom=0.05, left=0.7, right=0.845) 
k = 0
for i in range(10):
    for j in range(3):
      ax = plt.subplot(gs[i,j])
      img = vis_test_images[k]
      ax.imshow(img)
      prediction = model.predict(img.reshape(1,128, 128, 3))[0]
      true_label = vis_true_labels[k]
      k = k + 1
      ax.axis('off')
      ax.set_title("True Label: " + true_label + "\nPredicted: " + class_labels[np.argmax(prediction)])

plt.show()
